# Imports

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import mixed_precision
from tensorflow.keras.preprocessing import image

# Load Dataset

In [ ]:
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
batch_size = 64
img_height = 224
img_width = 224

In [ ]:
train_dataset, validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'data',
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)

class_names = train_dataset.class_names
print(class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# Create model and fit 

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

# Results for custom model

In [ ]:
def image_classification(model, img_path):
    
    # Load the image and resize it to the target size of the model (e.g., 224x224 for MobileNet)
    img = image.load_img(img_path, target_size=(224, 224))
    
    # Convert the image to a numpy array
    img_array = image.img_to_array(img)
    
    # Expand dimensions to match the shape the model expects (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = model.predict(img_array)

    # If you have a classification model, you might want to get the class with the highest probability
    predicted_class = np.argmax(predictions, axis=1)
    
    # Print the predicted class
    print(f'Predicted class: {predicted_class}')                        

# Create MobileNetV2 model

In [ ]:
from tensorflow.keras.models import Model

start_time = time.time()

preprocessing_input = layers.Input(shape=(img_height, img_width, 3))
x = layers.Rescaling(1./255)(preprocessing_input)
x = data_augmentation(x)  # Apply data augmentation
preprocessing = Model(inputs=preprocessing_input, outputs=x)

base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

base_model.summary()

input_tensor = preprocessing(preprocessing_input)
x = base_model(input_tensor, training=True)  # Apply base model
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Add pooling layer
x = layers.Dense(512, activation='relu')(x)  # Add dense layer
predictions = layers.Dense(len(class_names), activation='softmax')(x)  # Output layer

model = Model(inputs=preprocessing_input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
model.summary()
# Train the model
epochs = 20
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=epochs)

print("--- Execution time: %s seconds ---" % (time.time() - start_time))

model.save('tensorflow_mobilenet_v2_model.keras')

# Results for MobileNetV2 model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model = tf.keras.models.load_model('tensorflow_mobilenet_v2_model.keras')
image_classification(model, "data/butterfly_test.jpeg")
image_classification(model, "data/cat_test.jpeg")
image_classification(model, "data/chicken_test.jpeg")
image_classification(model, "data/cow_test.jpg")
image_classification(model, "data/dog_test.jpeg")
image_classification(model, "data/elephant_test.jpeg")
image_classification(model, "data/horse_test.jpg")
image_classification(model, "data/sheep_test.jpg")
image_classification(model, "data/spider_test.jpg")
image_classification(model, "data/squirrel_test.jpeg")

# Create EfficientNetB0

# Results for EfficientNetB0